In [2]:
import torch.nn as nn
import torch
class RMSNorm(nn.Module):

    def __init__(
        self,
        emb_dim,
        eps=1e-6,
        bias=False,
        qwen3_compatible=True,
    ):
        super().__init__()
        self.eps = eps
        self.qwen3_compatible = qwen3_compatible
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = (
            nn.Parameter(torch.zeros(emb_dim)) if bias
            else None
        )

    def forward(self, x):
        input_dtype = x.dtype

        if self.qwen3_compatible:
            x = x.to(torch.float32)

        variance = x.pow(2).mean(dim=-1, keepdim=True)
        norm_x = x * torch.rsqrt(variance + self.eps)
        norm_x = norm_x * self.scale

        if self.shift is not None:
            norm_x = norm_x + self.shift

        return norm_x.to(input_dtype)


In [3]:
nn.Linear(dtype=)

SyntaxError: expected argument value expression (3692773971.py, line 1)

In [ ]:
cfg = {
    "emb_dim": 40,
    "hidden_dim": 1,
    "dtype": torch.bfloat16
}

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.fc1 = nn.Linear(
            cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"],
            bias=False
        )
        self.fc2 = nn.Linear(
            cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"],
            bias=False
        )
        self.fc3 = nn.Linear(
            cfg["hidden_dim"], cfg["emb_dim"], dtype=cfg["dtype"],
            bias=False
        )

    def forward(self, x):
        x_fc1 = self.fc1(x)
        print(x_fc1.shape)
        x_fc2 = self.fc2(x)
        print(x_fc2.shape)
        
        # The non-linear activation function here is a SiLU function,           
        # which will be discussed later
        x = nn.functional.silu(x_fc1) * x_fc2
        print("before fc3: ",x)
        print("fc2: ",self.fc3.weight)
        return self.fc3(x)

network_a = FeedForward(cfg)

print(network_a)

x = torch.randn(40, dtype=cfg["dtype"])
network_a(x)

FeedForward(
  (fc1): Linear(in_features=40, out_features=1, bias=False)
  (fc2): Linear(in_features=40, out_features=1, bias=False)
  (fc3): Linear(in_features=1, out_features=40, bias=False)
)
torch.Size([1])
torch.Size([1])
before fc3:  tensor([0.0825], dtype=torch.bfloat16, grad_fn=<MulBackward0>)


tensor([-0.0825, -0.0649, -0.0381,  0.0708,  0.0801, -0.0786,  0.0464, -0.0026,
         0.0039,  0.0123, -0.0303, -0.0562, -0.0187,  0.0515, -0.0032, -0.0271,
         0.0767, -0.0187, -0.0476, -0.0718, -0.0084, -0.0212,  0.0791, -0.0245,
         0.0522, -0.0039,  0.0206,  0.0542, -0.0374, -0.0019,  0.0630, -0.0476,
        -0.0737, -0.0090,  0.0283, -0.0167,  0.0535,  0.0620, -0.0737, -0.0212],
       dtype=torch.bfloat16, grad_fn=<SqueezeBackward4>)

In [16]:
torch_temp = network_a.fc3.weight
torch_temp

Parameter containing:
tensor([[-0.4062],
        [-0.7344],
        [ 0.9766],
        [ 0.2422],
        [ 0.8125],
        [-0.3047],
        [-0.1172],
        [-0.5312],
        [-0.4219],
        [ 0.2500],
        [ 0.0859],
        [-0.3125],
        [-0.0859],
        [ 0.1016],
        [ 0.2344],
        [-0.8203],
        [ 0.4141],
        [-0.1875],
        [ 0.7031],
        [ 0.4922],
        [ 0.4297],
        [ 0.6172],
        [-0.7891],
        [ 0.8828],
        [ 0.4219],
        [ 0.3359],
        [ 0.9766],
        [ 0.0469],
        [-0.7500],
        [-0.7188],
        [-0.4375],
        [ 0.8906],
        [ 0.0938],
        [ 0.8047],
        [-0.7500],
        [-0.4844],
        [-0.6719],
        [ 0.3203],
        [-0.6484],
        [ 0.0000]], dtype=torch.bfloat16, requires_grad=True)

In [17]:
torch_1 = torch.randn(5)
torch_1

tensor([-0.8498,  0.5106, -0.2349,  0.9104, -2.0765])

In [19]:
torch_2 = torch.randn(1)
torch_2

tensor([1.1497])

In [21]:
torch_3 = torch_1 * torch_2 

In [22]:
torch_3

tensor([-0.9770,  0.5871, -0.2700,  1.0466, -2.3873])